In [4]:

from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [0]:

#Import all the required libraries
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint
from keras.layers.advanced_activations import PReLU
from keras.preprocessing import sequence, text
from keras.layers import SpatialDropout1D
from keras import metrics

#Load the training and test data
train = pd.read_csv('/content/drive/My Drive/web_scrap/data/train.csv')
test = pd.read_csv('/content/drive/My Drive/web_scrap/data/test.csv')

y_train = train.label.values
y_test = test.label.values

tk = text.Tokenizer(num_words=200000)
train.link_name = train.link_name.astype(str)
test.link_name = test.link_name.astype(str)
train.textdata = train.textdata.astype(str)
test.textdata = test.textdata.astype(str)

max_len = 80

tk.fit_on_texts(list(train.link_name.values) + list(train.textdata.values) + list(test.link_name.values) + list(
    test.textdata.values))
x_train_title = tk.texts_to_sequences(train.link_name.values)
x_train_title = sequence.pad_sequences(x_train_title, maxlen=max_len)

x_train_textdata_01 = tk.texts_to_sequences(train.textdata.values)
x_train_textdata_01 = sequence.pad_sequences(x_train_textdata_01, maxlen=max_len)

x_test_title_01 = tk.texts_to_sequences(test.link_name.values)
x_test_title_01 = sequence.pad_sequences(x_test_title_01, maxlen=max_len)

x_test_textdata_02 = tk.texts_to_sequences(test.textdata.values)
x_test_textdata_02 = sequence.pad_sequences(x_test_textdata_02, maxlen=max_len)

word_index = tk.word_index
ytrain_enc = np_utils.to_categorical(y_train)

lstm_check_model = Sequential()
lstm_check_model.add(Embedding(len(word_index) + 1, 300, input_length=80, dropout=0.2),)
lstm_check_model.add(LSTM(300, return_sequences=True, dropout=0.2, recurrent_dropout=0.2))

lstm_check_model.add(Dense(200))
lstm_check_model.add(PReLU())
lstm_check_model.add(SpatialDropout1D(0.2))

lstm_check_model.add(BatchNormalization())

lstm_check_model.add(Dense(200))
lstm_check_model.add(PReLU())
lstm_check_model.add(SpatialDropout1D(0.2))
lstm_check_model.add(BatchNormalization())

lstm_check_model.add(Flatten())

lstm_check_model.add(Dense(2))
lstm_check_model.add(Activation('softmax'))


lstm_check_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc' ])


checkpoint = ModelCheckpoint('/content/drive/My Drive/web_scrap/data/weights.h5', monitor='val_acc', save_best_only=True, verbose=2)

lstm_check_model.fit(x_train_title, y=ytrain_enc,
                 batch_size=128, epochs=20, verbose=2, validation_split=0.1,
                 shuffle=True, callbacks=[checkpoint])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:47: UserWarning: The `dropout` argument is no longer support in `Embedding`. You can apply a `keras.layers.SpatialDropout1D` layer right after the `Embedding` layer to get the same behavior.


Train on 4167 samples, validate on 463 samples
Epoch 1/20
 - 50s - loss: 0.9584 - acc: 0.6280 - val_loss: 0.2191 - val_acc: 0.9136

Epoch 00001: val_acc improved from -inf to 0.91361, saving model to /content/drive/My Drive/web_scrap/data/weights.h5
Epoch 2/20
 - 8s - loss: 0.1306 - acc: 0.9554 - val_loss: 0.2402 - val_acc: 0.9309

Epoch 00002: val_acc improved from 0.91361 to 0.93089, saving model to /content/drive/My Drive/web_scrap/data/weights.h5
Epoch 3/20
 - 8s - loss: 0.0369 - acc: 0.9890 - val_loss: 0.1450 - val_acc: 0.9676

Epoch 00003: val_acc improved from 0.93089 to 0.96760, saving model to /content/drive/My Drive/web_scrap/data/weights.h5
Epoch 4/20
 - 8s - loss: 0.0093 - acc: 0.9974 - val_loss: 0.1823 - val_acc: 0.9698

Epoch 00004: val_acc improved from 0.96760 to 0.96976, saving model to /content/drive/My Drive/web_scrap/data/weights.h5
Epoch 5/20
 - 8s - loss: 0.0055 - acc: 0.9986 - val_loss: 0.1549 - val_acc: 0.9719

Epoch 00005: val_acc improved from 0.96976 to 0.971